In [68]:
import sys
sys.path.insert(0, '../active_metric_learning/')
import faiss
import random
import numpy as np
import os
import forward_ranker.load_data as load_data
from forward_ranker.utils import print_message
from forward_ranker.utils import timestamp
from opts import get_opts_active_learning
from testing import load_model, transform_np_transformation

obj_reader = load_data.obj_reader
obj_writer = load_data.obj_writer

In [29]:
def load_without_filter(path, all_results):
    with open(path, "r") as f:
        for line in f:
            split_line = line.split("\t")
            qid = int(split_line[0])
            pid = int(split_line[1])
            if qid not in all_results:
                all_results[qid] = []
            all_results[qid].append(pid)

In [66]:
def load_ground_truth(path, all_qids, all_pids, all_results, n=100, sep="\t"):
    with open(path, "r") as f:
        for line in f:
            split_line = line.split(sep)
            qid = int(split_line[0])
            if qid not in all_qids:
                continue
            pid = int(split_line[1])
            if pid not in all_pids:
                continue
            if qid not in all_results:
                all_results[qid] = []
            if len(all_results[qid]) < n: 
                all_results[qid].append(pid)

In [58]:
def filter_data(all_results, all_qids, all_pids):
    new_results = {}
    for qid in all_qids:
        if qid not in all_results:
            print("{} not exist".format(qid))
        else:
            new_results[qid] = all_results[qid]

In [59]:
def write_data(all_results,output_path,n=100):
    for qid, pids in all_results.items():
        if len(pids) != n:
            print("Not enough pids for query {}. Only {} passages ranked.".format(qid, len(pids)))
        for i, pid in enumerate(pids):
            with open(output_path, "a") as f:
                f.write("{},{},{}\n".format(qid, pid, i + 1))

In [61]:
def add_particular_query(output_path,input_path,all_qids,all_pids,n=100):
    all_results = {}
    load_ground_truth(input_path, all_qids, all_pids, all_results)
    print(len(all_results))
    write_data(all_results,output_path,n)

In [40]:
pid_mapping = obj_reader("/datadrive/jianx/data/annoy/100_ance_passage_map.dict")
query_train_mapping = obj_reader("/datadrive/jianx/data/annoy/100_ance_query_train_map.dict")

In [41]:
all_pids = set(list(pid_mapping.values()))
all_qids = set(list(query_train_mapping.values()))

In [43]:
folder_path = "/datadrive/jianx/data/anserini/"
all_paths = os.listdir(folder_path)
all_results = {}
for path in all_paths:
    print(path)
    load_ground_truth(folder_path + path, all_qids, all_pids, all_results)
    print(len(all_results))

run.train7.tsv
62340
run.train0.tsv
124734
run.train3.tsv
186972
run.train2.tsv
248917
run.train6.tsv
311045
run.train1.tsv
373244
run.train4.tsv
435309
run.train5.tsv
497467
run.train8.tsv
502930


In [45]:
found_set = set(list(all_results.keys()))
diff_set = all_qids - found_set
print(len(diff_set))
print(diff_set)

9
{48509, 1078982, 140329, 502557, 205266, 129844, 522517, 197820, 56573}


In [64]:
xset = {1,2,3}
yset = {2}
print(xset.intersection(yset))

{2}


In [54]:
write_data(all_results,"/datadrive/ruohan/bm25/ground_truth/bm25_top100.csv",n=100)

Not enough pids for query 1081460. Only 20 passages ranked.
Not enough pids for query 403317. Only 40 passages ranked.
Not enough pids for query 409689. Only 66 passages ranked.
Not enough pids for query 445386. Only 12 passages ranked.
Not enough pids for query 413914. Only 32 passages ranked.
Not enough pids for query 1184859. Only 30 passages ranked.
Not enough pids for query 537503. Only 78 passages ranked.
Not enough pids for query 154430. Only 16 passages ranked.
Not enough pids for query 4826. Only 96 passages ranked.
Not enough pids for query 432576. Only 68 passages ranked.
Not enough pids for query 140937. Only 46 passages ranked.
Not enough pids for query 1182290. Only 13 passages ranked.
Not enough pids for query 535776. Only 88 passages ranked.
Not enough pids for query 1172686. Only 14 passages ranked.
Not enough pids for query 535851. Only 5 passages ranked.
Not enough pids for query 423562. Only 77 passages ranked.
Not enough pids for query 498824. Only 84 passages rank

In [60]:
add_particular_query("/datadrive/ruohan/bm25/ground_truth/bm25_top100.csv",
                     "/datadrive/jianx/data/anserini/run.train9.tsv",all_qids,all_pids,n=100)

7


In [63]:
add_particular_query("/datadrive/ruohan/bm25/ground_truth/bm25_top100.csv",
                     "/datadrive/jianx/data/anserini/run.train9.tsv",set([140329]),all_pids,n=100)

1


In [67]:
add_particular_query("/datadrive/ruohan/bm25/ground_truth/bm25_top100.csv",
                     "/datadrive/jianx/data/results/all_search_rankings_100_100_flat.csv",set([197820]),all_pids,n=100)

1


In [30]:
folder_path = "/datadrive/jianx/data/anserini/"
all_paths = os.listdir(folder_path)
all_results_without = {}
for path in all_paths:
    print(path)
    load_without_filter(folder_path + path, all_results_without)
    print(len(all_results_without))

run.train7.tsv
99997
run.train3.tsv
199997
run.train2.tsv
299993
run.train6.tsv
399991
run.train1.tsv
499988
run.train4.tsv
599987
run.train5.tsv
699987
run.train8.tsv
708716


In [22]:
print(len(all_results))

440536


In [21]:
print(len(all_qids))

502939


In [27]:
len(all_pids)

8841823

In [ ]:
print(len(all_results))

# Alternative baseline model: 
* Treat passage as "long" query 
* Treat query as "short" passage
<p> Only need this for 200,000 passages and all candidates queries

In [69]:
query_train_mapping = obj_reader("/datadrive/jianx/data/annoy/100_ance_query_train_map.dict")
all_qids = set(list(query_train_mapping.values()))

In [71]:
test_path = "/datadrive/ruohan/final_train_test_data/ance_testing_rank100_nqueries50000_npassages20000_bm25.csv"
test_passages = set()
with open(test_path, "r") as f:
    for line in f:
        split_line = line.split(",")
        pid = int(split_line[0])
        test_passages.add(pid)

In [72]:
print(len(all_qids))
print(len(test_passages))

502939
20000


In [93]:
train_path = "/datadrive/ruohan/final_train_test_data/ance_training_rank100_nqueries50000_npassages200000_bm25.csv"
train_passages = set()
with open(train_path, "r") as f:
    for line in f:
        split_line = line.split(",")
        pid = int(split_line[0])
        train_passages.add(pid)
        if len(train_passages) >= 20000:
            break

In [94]:
len(train_passages)

20000

In [73]:
passage_text_path = "/datadrive/jianx/data/collection.tsv"
query_text_path = "/datadrive/jianx/data/queries.train.tsv"

In [81]:
def write_selected(input_path, output_path, select_set):
    output_dict = {}
    with open(input_path, "r") as f:
        for line in f:
            split_line = line.split("\t")
            idx = int(split_line[0])
            if idx in select_set:
                output_dict[idx] = split_line[1]
    if len(select_set) != len(output_dict):
        print("Only {}/{} have raw text".format(len(output_dict), len(select_set)))
    else:
        for idx, text in output_dict.items():
            with open(output_path, "a") as f:
                f.write("{}\t{}".format(idx, text))

In [82]:
test_folder = "/datadrive/ruohan/bm25/test_data/"
write_selected(passage_text_path, test_folder + "test_passage.tsv", test_passages)

In [83]:
write_selected(query_text_path, test_folder + "all_queries.tsv", all_qids)

In [95]:
train_folder = "/datadrive/ruohan/bm25/train_data/"

In [96]:
write_selected(passage_text_path, train_folder + "train_passage.tsv", train_passages)

### Generate test data with pid,qid,rank

In [97]:
baseline_path = "/home/jianx/anserini/runs/run.msmarco-query.tsv"
output_baseline_path = "/datadrive/ruohan/final_train_test_data/bm25_inverse_index_baseline.csv"
ground_truth_path = "/datadrive/ruohan/bm25/ground_truth/bm25_top100.csv"

In [91]:
def generate_test_data(baseline_path, output_baseline_path, ground_truth_path):
    all_results = {}
    with open(ground_truth_path, "r") as f:
        for line in f:
            line_split = line.split(",")
            qid = int(line_split[0])
            pid = int(line_split[1])
            rank = int(line_split[2])
            if qid not in all_results.keys():
                all_results[qid] = {}
            all_results[qid][pid] = rank
    with open(baseline_path, "r") as f:
        for line in f:
            line_split = line.split("\t")
            pid = int(line_split[0])
            qid = int(line_split[1])
            rank = all_results[qid].get(pid, 0)
            with open(output_baseline_path, "a") as f:
                f.write("{},{},{}\n".format(pid, qid, rank))

In [92]:
generate_test_data(baseline_path, output_baseline_path, ground_truth_path)

In [101]:
generate_test_data("/home/jianx/anserini/runs/bm25_test_k1_0.8_b_0.8.tsv", 
                   "/datadrive/ruohan/bm25/results/bm25_test_results_k1_0.8_b_0.8.csv", ground_truth_path)

In [98]:
all_results = {}
with open(ground_truth_path, "r") as f:
    for line in f:
        line_split = line.split(",")
        qid = int(line_split[0])
        pid = int(line_split[1])
        rank = int(line_split[2])
        if qid not in all_results.keys():
            all_results[qid] = {}
        all_results[qid][pid] = rank

In [99]:
obj_writer(all_results, "/datadrive/ruohan/bm25/ground_truth/bm25_top100.dict")

In [100]:
all_results = obj_reader("/datadrive/ruohan/bm25/ground_truth/bm25_top100.dict")

# Compare bm25 baseline performance with ANCE baseline